In this notebook, we resolve the problem
$$
\min_{u, u_0} \ \frac{1}{2}\int_{0}^{T}\int_{\Omega} |u-u_R|^2\textrm{d}x\textrm{d}t+\frac{\alpha}{2}\int_{\Omega} |u_0|^2\textrm{d}x
$$
subject to Heat equation
$$
\dfrac{\partial u}{\partial t}-\nu\Delta u = 0  \qquad \mathrm{in } \ \Omega\times (0,T)
$$
with initial and Dirichlet boundary conditions
$$
\begin{split}
u&= 0  \qquad \mathrm{on} \ \partial \Omega \times (0,T)\\
u(\cdot,0) &= u_0  \qquad \mathrm{on} \ \Omega \\\end{split}
$$
where $u_R(x,y,t)=\exp(-8\pi^2 t)\sin(2\pi x)\sin(2\pi y)$ and $u_0\in L^2 (\Omega)$

In [1]:
from __future__ import print_function
from dolfin import *
from dolfin_adjoint import *
import numpy as np

set_log_level(LogLevel.WARNING)
# Defining mesh and FEM spaces
n=20
mesh = RectangleMesh(Point(0,0),Point(1,1),n,n,diagonal="left")
V = FunctionSpace(mesh, "CG", 1)
tol = 1e-14
# Defining bounday conditions
u_R = Expression('0.0', degree=2)
def boundary(x):
    return x[0] < DOLFIN_EPS or x[0] > 1.0 - DOLFIN_EPS or x[1] < DOLFIN_EPS or x[1] > 1.0 - DOLFIN_EPS
uD = Constant(0.0)
bc = DirichletBC(V, uD, boundary)
# Resolving the variational formulation    
dt = 0.0025
u0 = interpolate(Expression("0.0", degree=4), V)
u_prev = u0.copy(deepcopy=True) 
u_next = u0.copy(deepcopy=True)
T = 0.1
t = 0.0
v = TestFunction(V)
states = [u0.copy(deepcopy=True)]
times = [float(t)]
timestep = 0
while abs(T-t)>=tol :
    # print("Solving for t == %s" % (t + dt))
    F = inner((u_next - u_prev) / Constant(dt), v) * dx + inner(grad(u_next), grad(v)) * dx
    solve(F == 0, u_next, bc, annotate=True) # This is how dolfin-adjoint records the dependency of the control in each timestep
    u_prev.assign(u_next, annotate=True)
    t += dt
    timestep += 1
    states.append(u_next.copy(deepcopy=True, annotate=True))
    times.append(float(t))
m = Control(u0)
steps=len(times)
# Defining reference solution
for j in np.arange(0,steps):
    w=interpolate(Expression("exp(-8*pi*pi*t)*sin(2*pi*x[0])*sin(2*pi*x[1])",t=times[j],degree=1), V)
    if j==0:
        reference = [w.copy(deepcopy=True)]
    else:
        reference.append(w.copy(deepcopy=True, annotate=False))
combined = zip(times,states,reference)
alpha = Constant(0)
# and the minimizing problem
J = assemble(1/dt*sum(inner(true-computed,true-computed)*dx for (time,true,computed) in combined if time >= dt) + alpha*inner(u0,u0)*dx)
RJ = ReducedFunctional(J,m)

u0_opt = minimize(RJ,tol=1.0e-10, options={"ftol": 1e-7})

Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FF

In [2]:
u0_opt.rename('u','u')
u_p = u0_opt.copy(deepcopy=True)
u_n = u0_opt.copy(deepcopy=True)
# Finally, we obtain the optimal state in each timestep
T = 0.1
t = 0.0
v = TestFunction(V)
sol = [u0_opt.copy(deepcopy=True)]
file = File('TimeTest/sol.pvd')
file << (u0_opt,0)
while abs(T-t)>=tol :
    print("Solving for t == %s" % (t + dt))
    F = inner((u_n - u_p) / Constant(dt), v) * dx + inner(grad(u_n), grad(v)) * dx
    solve(F == 0, u_n, bc)
    u_p.assign(u_n)
    t += dt
    timestep += 1
    sol.append(u_next.copy(deepcopy=True))
    u_n.rename('u','u')
    file << (u_n,t)

Solving for t == 0.0025
Solving for t == 0.005
Solving for t == 0.0075
Solving for t == 0.01
Solving for t == 0.0125
Solving for t == 0.015000000000000001
Solving for t == 0.0175
Solving for t == 0.02
Solving for t == 0.0225
Solving for t == 0.024999999999999998
Solving for t == 0.027499999999999997
Solving for t == 0.029999999999999995
Solving for t == 0.032499999999999994
Solving for t == 0.034999999999999996
Solving for t == 0.0375
Solving for t == 0.04
Solving for t == 0.0425
Solving for t == 0.045000000000000005
Solving for t == 0.04750000000000001
Solving for t == 0.05000000000000001
Solving for t == 0.05250000000000001
Solving for t == 0.055000000000000014
Solving for t == 0.057500000000000016
Solving for t == 0.06000000000000002
Solving for t == 0.06250000000000001
Solving for t == 0.06500000000000002
Solving for t == 0.06750000000000002
Solving for t == 0.07000000000000002
Solving for t == 0.07250000000000002
Solving for t == 0.07500000000000002
Solving for t == 0.077500000000

In [1]:
import os
os.system('python3 TimeTest.py')

0